In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
np.random.seed(2)

In [2]:
train=pd.read_csv('train_upd.csv')
train.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0  1090595862       1      2018         12       15         0       50   
1  4826375369       1      2018         12        7        20        5   
2  3607540856       1      2018         12        5        16       35   
3  1245076813       0      2018         12       16        21       60   
4   404129745       1      2018         12        4         6       60   

   subscriber_count  web_browsing_total_bytes  video_total_bytes  \
0               152                       315              59012   
1                54                      1871                334   
2               277                       887                945   
3                51                      8577                494   
4               117                      2084               1056   

            ...            email_total_bytes  weather_total_bytes  \
0           ...                           10                   33   
1           ...                           22                   20   
2           ...                           14                   91   
3           ...                           10                   63   
4           ...                           67                   59   

   media_total_bytes  mms_total_bytes  others_total_bytes  beam_direction  \
0                 46               39                 103              69   
1                  7               10                  21              93   
2                 49              147                 159              77   
3                 42               75                  35              95   
4                 91              197                 316             112   

   cell_range  tilt  ran_vendor         Congestion_Type  
0           6     2    ERICSSON  4G_BACKHAUL_CONGESTION  
1           7     5       NOKIA                      NC  
2           6     3    ERICSSON  3G_BACKHAUL_CONGESTION  
3           7     2      HUAWEI  3G_BACKHAUL_CONGESTION  
4           3     5      HUAWEI       4G_RAN_CONGESTION  

[5 rows x 39 columns]

In [3]:
normal_cols= ['par_day', 'par_hour', 'par_min', 'subscriber_count', 'beam_direction', 'cell_range','tilt', 'ran_vendor']

In [4]:
train['Congestion_Type'].unique()

array(['4G_BACKHAUL_CONGESTION', 'NC', '3G_BACKHAUL_CONGESTION',
       '4G_RAN_CONGESTION'], dtype=object)

In [5]:
train['ran_vendor'].unique()

array(['ERICSSON', 'NOKIA', 'HUAWEI'], dtype=object)

In [6]:
lb1=LabelEncoder()
train['ran_vendor']=lb1.fit_transform(train['ran_vendor'])

In [7]:
lb1.classes_

array(['ERICSSON', 'HUAWEI', 'NOKIA'], dtype=object)

In [8]:
lb2=LabelEncoder()
train['Congestion_Type']=lb2.fit_transform(train['Congestion_Type'])

In [9]:
lb2.classes_

array(['3G_BACKHAUL_CONGESTION', '4G_BACKHAUL_CONGESTION',
       '4G_RAN_CONGESTION', 'NC'], dtype=object)

In [10]:
Y=train['Congestion_Type'].values

In [11]:
X=train.drop(['Congestion_Type','cell_name'],axis=1).values

In [12]:
X_scaled= scale(X)

E:\anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [13]:
X_train,X_val,Y_train, Y_val=train_test_split(X_scaled,Y,test_size=0.3)

In [14]:
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

In [15]:
train.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0  1090595862       1      2018         12       15         0       50   
1  4826375369       1      2018         12        7        20        5   
2  3607540856       1      2018         12        5        16       35   
3  1245076813       0      2018         12       16        21       60   
4   404129745       1      2018         12        4         6       60   

   subscriber_count  web_browsing_total_bytes  video_total_bytes  \
0               152                       315              59012   
1                54                      1871                334   
2               277                       887                945   
3                51                      8577                494   
4               117                      2084               1056   

        ...         email_total_bytes  weather_total_bytes  media_total_bytes  \
0       ...                        10                   33                 46   
1       ...                        22                   20                  7   
2       ...                        14                   91                 49   
3       ...                        10                   63                 42   
4       ...                        67                   59                 91   

   mms_total_bytes  others_total_bytes  beam_direction  cell_range  tilt  \
0               39                 103              69           6     2   
1               10                  21              93           7     5   
2              147                 159              77           6     3   
3               75                  35              95           7     2   
4              197                 316             112           3     5   

   ran_vendor  Congestion_Type  
0           0                1  
1           2                3  
2           0                0  
3           1                0  
4           1                2  

[5 rows x 39 columns]

In [16]:
RFC=RandomForestClassifier(n_estimators=300)
RFC.fit(X_train,Y_train)
pred=RFC.predict(X_val)
matthews_corrcoef(pred,Y_val)

0.67061399968059998

In [17]:
mlp= MLPClassifier()
mlp.fit(X_train, Y_train)
pred_mlp= mlp.predict(X_val)
matthews_corrcoef(pred_mlp, Y_val)

0.70303749578479058

In [18]:
ada= AdaBoostClassifier()
ada.fit(X_train, Y_train)
pred_ada= ada.predict(X_val)
matthews_corrcoef(pred_ada, Y_val)

0.63491536630074286

In [19]:
lr= LogisticRegression()
lr.fit(X_train, Y_train)
pred_lr= lr.predict(X_val)
matthews_corrcoef(pred_lr, Y_val)

0.70339025885845563

In [21]:
! pip install xgboost

You are using pip version 9.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [22]:
from xgboost import XGBClassifier
XG = XGBClassifier(n_estimators=500)
XG.fit(X_train, Y_train)
yn_pred = XG.predict(X_val)

from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(Y_val, yn_pred)
#train_scores, valid_scores=validation_curve(XG, Xn, yn, "n-estimators", [50,100,200,400,500])

0.72915653855208828

In [23]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier,BaggingClassifier,RandomTreesEmbedding,IsolationForest,VotingClassifier
from sklearn.linear_model import LogisticRegression


GRA=GradientBoostingClassifier(n_estimators=500)
GRA.fit(X_train,Y_train)
pred_GRA=GRA.predict(X_val)
print(matthews_corrcoef(Y_val,pred_GRA))

0.723531120379


# feature engineering

# x ==> log(1+x)

In [24]:
train=pd.read_csv('train_upd.csv')
train.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0  1090595862       1      2018         12       15         0       50   
1  4826375369       1      2018         12        7        20        5   
2  3607540856       1      2018         12        5        16       35   
3  1245076813       0      2018         12       16        21       60   
4   404129745       1      2018         12        4         6       60   

   subscriber_count  web_browsing_total_bytes  video_total_bytes  \
0               152                       315              59012   
1                54                      1871                334   
2               277                       887                945   
3                51                      8577                494   
4               117                      2084               1056   

            ...            email_total_bytes  weather_total_bytes  \
0           ...                           10                   33   
1           ...                           22                   20   
2           ...                           14                   91   
3           ...                           10                   63   
4           ...                           67                   59   

   media_total_bytes  mms_total_bytes  others_total_bytes  beam_direction  \
0                 46               39                 103              69   
1                  7               10                  21              93   
2                 49              147                 159              77   
3                 42               75                  35              95   
4                 91              197                 316             112   

   cell_range  tilt  ran_vendor         Congestion_Type  
0           6     2    ERICSSON  4G_BACKHAUL_CONGESTION  
1           7     5       NOKIA                      NC  
2           6     3    ERICSSON  3G_BACKHAUL_CONGESTION  
3           7     2      HUAWEI  3G_BACKHAUL_CONGESTION  
4           3     5      HUAWEI       4G_RAN_CONGESTION  

[5 rows x 39 columns]

In [25]:
cols=[ 'subscriber_count', 'web_browsing_total_bytes',
       'video_total_bytes', 'social_ntwrking_bytes',
       'cloud_computing_total_bytes', 'web_security_total_bytes',
       'gaming_total_bytes', 'health_total_bytes', 'communication_total_bytes',
       'file_sharing_total_bytes', 'remote_access_total_bytes',
       'photo_sharing_total_bytes', 'software_dwnld_total_bytes',
       'marketplace_total_bytes', 'storage_services_total_bytes',
       'audio_total_bytes', 'location_services_total_bytes',
       'presence_total_bytes', 'advertisement_total_bytes',
       'system_total_bytes', 'voip_total_bytes', 'speedtest_total_bytes',
       'email_total_bytes', 'weather_total_bytes', 'media_total_bytes',
       'mms_total_bytes', 'others_total_bytes']
for each in cols:
    train[each]=np.log(train[each]+1)

In [26]:
lb1=LabelEncoder()
train['ran_vendor']=lb1.fit_transform(train['ran_vendor'])
lb2=LabelEncoder()
train['Congestion_Type']=lb2.fit_transform(train['Congestion_Type'])
X_scaled1=train.drop(['cell_name', 'Congestion_Type', 'par_year', 'par_month'], axis=1)
X_scaled1=(X_scaled1-X_scaled1.mean())/X_scaled1.std()
Y1=train['Congestion_Type'].values
X_train1,X_val1,Y_train1, Y_val1=train_test_split(X_scaled1,Y1,test_size=0.3)

In [27]:
mlp= MLPClassifier()
mlp.fit(X_train1, Y_train1)
pred_mlp1= mlp.predict(X_val1)
matthews_corrcoef(pred_mlp1, Y_val1)

0.68513128514548716

In [28]:
lr= LogisticRegression()
lr.fit(X_train1, Y_train1)
pred_lr1= lr.predict(X_val1)
matthews_corrcoef(pred_lr1, Y_val1)

0.67498968418882099

# span of the day and day of the week and one hot encoding

In [29]:
train=pd.read_csv('train_upd.csv')
train.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0  1090595862       1      2018         12       15         0       50   
1  4826375369       1      2018         12        7        20        5   
2  3607540856       1      2018         12        5        16       35   
3  1245076813       0      2018         12       16        21       60   
4   404129745       1      2018         12        4         6       60   

   subscriber_count  web_browsing_total_bytes  video_total_bytes  \
0               152                       315              59012   
1                54                      1871                334   
2               277                       887                945   
3                51                      8577                494   
4               117                      2084               1056   

            ...            email_total_bytes  weather_total_bytes  \
0           ...                           10                   33   
1           ...                           22                   20   
2           ...                           14                   91   
3           ...                           10                   63   
4           ...                           67                   59   

   media_total_bytes  mms_total_bytes  others_total_bytes  beam_direction  \
0                 46               39                 103              69   
1                  7               10                  21              93   
2                 49              147                 159              77   
3                 42               75                  35              95   
4                 91              197                 316             112   

   cell_range  tilt  ran_vendor         Congestion_Type  
0           6     2    ERICSSON  4G_BACKHAUL_CONGESTION  
1           7     5       NOKIA                      NC  
2           6     3    ERICSSON  3G_BACKHAUL_CONGESTION  
3           7     2      HUAWEI  3G_BACKHAUL_CONGESTION  
4           3     5      HUAWEI       4G_RAN_CONGESTION  

[5 rows x 39 columns]

In [30]:
val = []
for item in train['par_hour'] :
    value=0
    if(item<7 and item>=1):
        value = 0
    elif (item<11 and item>=7) :
        value = 1
    elif (item<16 and item>=11) :
        value = 2
    elif (item<20 and item>=16) :
        value = 3
    elif (item>=20) :
        value = 4
    val.append(value)

In [31]:
day_of_week= []
for item in train['par_day']:
    day_of_week.append(item%7)

In [32]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train['ran_vendor'].values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


In [33]:
train['span_of_day']= val
train['day_of_week']= day_of_week
train['ERICSSON']= onehot_encoded[:, 0]
train['HUAWEI']= onehot_encoded[:, 1]
train['NOKIA']= onehot_encoded[:, 2]

In [34]:
train.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0  1090595862       1      2018         12       15         0       50   
1  4826375369       1      2018         12        7        20        5   
2  3607540856       1      2018         12        5        16       35   
3  1245076813       0      2018         12       16        21       60   
4   404129745       1      2018         12        4         6       60   

   subscriber_count  web_browsing_total_bytes  video_total_bytes  ...    \
0               152                       315              59012  ...     
1                54                      1871                334  ...     
2               277                       887                945  ...     
3                51                      8577                494  ...     
4               117                      2084               1056  ...     

   beam_direction  cell_range  tilt  ran_vendor         Congestion_Type  \
0              69           6     2    ERICSSON  4G_BACKHAUL_CONGESTION   
1              93           7     5       NOKIA                      NC   
2              77           6     3    ERICSSON  3G_BACKHAUL_CONGESTION   
3              95           7     2      HUAWEI  3G_BACKHAUL_CONGESTION   
4             112           3     5      HUAWEI       4G_RAN_CONGESTION   

   span_of_day  day_of_week  ERICSSON  HUAWEI  NOKIA  
0            0            1       1.0     0.0    0.0  
1            4            0       0.0     0.0    1.0  
2            3            5       1.0     0.0    0.0  
3            4            2       0.0     1.0    0.0  
4            0            4       0.0     1.0    0.0  

[5 rows x 44 columns]

In [35]:
lb1=LabelEncoder()
train['ran_vendor']=lb1.fit_transform(train['ran_vendor'])
lb2=LabelEncoder()
train['Congestion_Type']=lb2.fit_transform(train['Congestion_Type'])
X_scaled1=train.drop(['cell_name', 'Congestion_Type', 'par_year', 'par_month'], axis=1)
X_scaled1=(X_scaled1-X_scaled1.mean())/X_scaled1.std()
Y1=train['Congestion_Type'].values
X_train1,X_val1,Y_train1, Y_val1=train_test_split(X_scaled1,Y1,test_size=0.3)

In [36]:
mlp= MLPClassifier()
mlp.fit(X_train1, Y_train1)
pred_mlp1= mlp.predict(X_val1)
matthews_corrcoef(pred_mlp1, Y_val1)

0.69768073456660173

In [37]:
lr= LogisticRegression()
lr.fit(X_train1, Y_train1)
pred_lr1= lr.predict(X_val1)
matthews_corrcoef(pred_lr1, Y_val1)

0.70443142668359993

# final model

In [65]:
train=pd.read_csv('train_upd.csv')
train.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0  1090595862       1      2018         12       15         0       50   
1  4826375369       1      2018         12        7        20        5   
2  3607540856       1      2018         12        5        16       35   
3  1245076813       0      2018         12       16        21       60   
4   404129745       1      2018         12        4         6       60   

   subscriber_count  web_browsing_total_bytes  video_total_bytes  \
0               152                       315              59012   
1                54                      1871                334   
2               277                       887                945   
3                51                      8577                494   
4               117                      2084               1056   

            ...            email_total_bytes  weather_total_bytes  \
0           ...                           10                   33   
1           ...                           22                   20   
2           ...                           14                   91   
3           ...                           10                   63   
4           ...                           67                   59   

   media_total_bytes  mms_total_bytes  others_total_bytes  beam_direction  \
0                 46               39                 103              69   
1                  7               10                  21              93   
2                 49              147                 159              77   
3                 42               75                  35              95   
4                 91              197                 316             112   

   cell_range  tilt  ran_vendor         Congestion_Type  
0           6     2    ERICSSON  4G_BACKHAUL_CONGESTION  
1           7     5       NOKIA                      NC  
2           6     3    ERICSSON  3G_BACKHAUL_CONGESTION  
3           7     2      HUAWEI  3G_BACKHAUL_CONGESTION  
4           3     5      HUAWEI       4G_RAN_CONGESTION  

[5 rows x 39 columns]

In [66]:
lb1=LabelEncoder()
train['ran_vendor']=lb1.fit_transform(train['ran_vendor'])
lb2=LabelEncoder()
train['Congestion_Type']=lb2.fit_transform(train['Congestion_Type'])
X_scaled1=train.drop(['cell_name', 'Congestion_Type', 'par_year', 'par_month', 'par_min', '4G_rat'], axis=1)
X_scaled1=(X_scaled1-X_scaled1.mean())/X_scaled1.std()
Y1=train['Congestion_Type'].values
X_train,X_val,Y_train, Y_val=train_test_split(X_scaled1,Y1,test_size=0.3)

In [40]:
GRA=GradientBoostingClassifier(n_estimators=500)
GRA.fit(X_train,Y_train)
pred_GRA=GRA.predict(X_val)
print(matthews_corrcoef(Y_val,pred_GRA))

0.720906111936


In [41]:
LR=LogisticRegression(C=1,penalty='l1',tol=0.0001,max_iter=300)
LR.fit(X_train,Y_train)
pred_LR=LR.predict(X_val)
print(matthews_corrcoef(Y_val,pred_LR))

0.703464497926


In [42]:
mlp= MLPClassifier(alpha=1,max_iter=300)
mlp.fit(X_train, Y_train)
pred_mlp= mlp.predict(X_val)
matthews_corrcoef(pred_mlp, Y_val)

0.71961369593879587

In [43]:
XG = XGBClassifier(n_estimators=500)
XG.fit(X_train, Y_train)
pred_XG = XG.predict(X_val)
matthews_corrcoef(pred_XG, Y_val)

0.72722601601130865

In [44]:
votingC=VotingClassifier([('GRA',GRA),('MLP',mlp),('LR',LR),('XG',XG)],voting='soft')
votingC.fit(X_train,Y_train)
pred=votingC.predict(X_val)
print(matthews_corrcoef(Y_val,pred))
confusion_matrix(Y_val,pred)

0.733347455792


array([[4718,  547,    0,  737],
       [ 793, 3971, 1082,    3],
       [  26, 1017, 4711,    0],
       [ 510,    0,    0, 5453]], dtype=int64)

In [45]:
test=pd.read_csv('test.csv')
test.head()

cell_name  4G_rat  par_year  par_month  par_day  par_hour  par_min  \
0     160261       0      2018         12       16        22       55   
1     453651       0      2018         12       23         9       20   
2     508631       1      2018         12       24         9       30   
3     662615       0      2018         12       28         0       35   
4     735573       0      2018         12       14         0       40   

   subscriber_count  web_browsing_total_bytes  video_total_bytes     ...      \
0               101                      1140               2799     ...       
1                44                      7505                473     ...       
2              2037                        66              11327     ...       
3                25                       970               4246     ...       
4              1494                      1101               2948     ...       

   speedtest_total_bytes  email_total_bytes  weather_total_bytes  \
0                     18                 18                   16   
1                     17                 11                   57   
2                     20                 21                  267   
3                     10                 10                    6   
4                    133                  3                   87   

   media_total_bytes  mms_total_bytes  others_total_bytes  beam_direction  \
0                167              103                  20              62   
1                 51               25                 493              69   
2                 30              242                  39              61   
3                 12               11                  26              70   
4                 69              290                  38              61   

   cell_range  tilt  ran_vendor  
0           5     4       NOKIA  
1           2     4       NOKIA  
2           2     3    ERICSSON  
3           3     3    ERICSSON  
4           7     2       NOKIA  

[5 rows x 38 columns]

In [46]:
test.shape

(26305, 38)

In [49]:
lb1=LabelEncoder()
test['ran_vendor']=lb1.fit_transform(test['ran_vendor'])
X_scaled1_test=test.drop(['cell_name', 'par_year', 'par_month', 'par_min', '4G_rat'], axis=1)
X_scaled1_test=(X_scaled1_test-X_scaled1_test.mean())/X_scaled1_test.std()

In [51]:
# Training on all train dataset and then predicting on test dataset
votingC.fit(X_scaled1, Y1)

VotingClassifier(estimators=[('GRA', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1,...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [52]:
y_pred=votingC.predict(X_scaled1_test)

In [67]:
y_pred=lb2.inverse_transform(y_pred)

In [54]:
y_pred.shape

(26305,)

In [72]:
submission = pd.DataFrame({
        "cell_name": test["cell_name"],
        "Congestion_Type": y_pred
    })

submission= submission[["cell_name", "Con"]]

In [73]:
submission.to_csv('prediction.csv',index=False)

In [68]:
y_pred

array(['3G_BACKHAUL_CONGESTION', '4G_RAN_CONGESTION', '4G_RAN_CONGESTION',
       ..., '4G_BACKHAUL_CONGESTION', '4G_BACKHAUL_CONGESTION',
       '3G_BACKHAUL_CONGESTION'], dtype=object)